In [1]:
import time
import datetime
import pandas as pd
import numpy as np
import typing as tp
import re
import matplotlib.pyplot as plt
import folium
import requests
import os
import gc
import seaborn as sns

In [ ]:
"""
1. 파일 경로 및 I/O 옵션
- filepath_or_buffer: 파일 경로 또는 파일 객체 (URL, 파일 경로, StringIO 등).

2. 구분자 및 열 처리 옵션
- sep: 열을 구분하는 문자열. 기본값은 ',' (예: '\t'는 탭 구분).
- delimiter: sep과 동일하지만 명확히 구분자를 지정할 때 사용.
- header: 열 이름이 있는 행의 인덱스. 기본값은 0 (첫 번째 행). None일 경우 첫 행을 데이터로 간주.
- names: 열 이름을 리스트로 지정. 파일에 헤더가 없을 때 유용.
- index_col: 인덱스로 사용할 열을 지정 (기본값 None).
- usecols: 특정 열만 읽기 위해 열 이름 또는 인덱스를 리스트나 호출 가능한 객체로 지정.

3. 데이터 유형 및 변환 옵션
- dtype: 열의 데이터 유형을 지정하는 딕셔너리 (예: { 'column_name': int }).
- converters: 각 열의 데이터를 변환하기 위한 함수 딕셔너리 (예: { 'column_name': function }).
- true_values: True로 간주할 값들의 리스트.
- false_values: False로 간주할 값들의 리스트.

4. 결측값 처리 옵션
- na_values: 결측값으로 처리할 값의 리스트 또는 딕셔너리.
- keep_default_na: 기본 결측값을 유지할지 여부 (기본값: True).
- na_filter: 결측값을 확인하지 않도록 설정 (대규모 데이터 처리 시 속도 향상 가능).
- skip_blank_lines: 빈 줄을 무시할지 여부 (기본값: True).

5. 행 건너뛰기 및 제한 옵션
- skiprows: 읽지 않고 건너뛸 행의 수나 리스트.
- nrows: 읽을 행의 수. 지정된 행까지만 읽음.
- skipfooter: 파일 끝에서 건너뛸 행 수 (기본값: 0).

6. 데이터 파싱 옵션
- parse_dates: 날짜 형식으로 파싱할 열을 리스트로 지정.
- date_parser: 사용자 정의 날짜 파싱 함수.
- dayfirst: 날짜가 DD/MM 형식인지 여부 (기본값: False).
- infer_datetime_format: 날짜 형식을 자동 추론할지 여부 (True일 경우 속도 향상 가능).

7. 텍스트 및 인코딩 옵션
- encoding: 파일의 인코딩을 지정 (예: 'utf-8', 'latin1').
- encoding_errors: 인코딩 오류 처리 방법 (예: 'ignore', 'strict').
- lineterminator: 행을 구분하는 문자열 (기본값: 자동 감지).
- quotechar: 값을 묶는 데 사용할 문자 (기본값: '"').
- escapechar: 구분 기호나 인용 부호가 포함된 필드를 읽을 때 사용하는 이스케이프 문자.

8. 메모리 및 성능 옵션
- chunksize: 한 번에 읽을 행 수 (대규모 데이터 처리 시 유용).
- iterator: True일 경우 데이터를 일부씩 읽을 수 있는 TextFileReader 객체 반환.
- memory_map: True로 설정하면 메모리 맵 파일을 사용하여 성능 개선.
- low_memory: 메모리 사용량을 줄이는 옵션 (기본값: True).

9. 기타 옵션
- comment: 주석으로 간주할 문자 (해당 문자가 있는 줄은 무시됨).
- verbose: True로 설정하면 파일 읽기 과정의 정보를 출력.
- error_bad_lines (Deprecated): 잘못된 행을 건너뛰기 (기본값: False).
- warn_bad_lines (Deprecated): 잘못된 행이 있을 때 경고 발생 (기본값: True).
- delim_whitespace: True일 경우 공백을 구분 기호로 사용.

10. 압축 옵션
- compression: 압축된 파일을 읽을 때 압축 유형 지정 (infer, gzip, bz2, zip, xz 중 선택).
- thousands: 숫자에서 천 단위 구분 기호를 제거하기 위한 옵션.
"""


In [7]:
data = pd.read_csv("data/big_df.csv", encoding = 'utf-8', iterator = True, chunksize = 1000)
chunk = data.get_chunk(1000)
chunk.head()

,Unnamed: 0,StartTime,UseTimeM,UseDistM,Birth,Gender,StartId,EndId,StartDate,Id,Addr1,Addr2,Lat,Lon
0,0,0.0,0,0.0,40,ETC,ST-2127,ST-2127,2023-01-01,ST-2127,서울특별시 강북구 번동 430-76,NaN,37.635391,127.034554
1,1,0.0,2,0.0,20,ETC,ST-1121,ST-2699,2023-01-01,ST-1121,서울특별시 노원구 상계동 335-4,노원역1번출구,37.656200,127.063622
2,2,0.0,0,0.0,30,M,ST-1590,ST-1590,2023-01-01,ST-1590,서울특별시 송파구 삼학사로 136,석촌호수 서호사거리,37.506748,127.098831
3,3,0.0,2,0.0,60,M,ST-1841,ST-2740,2023-01-01,ST-1841,서울특별시 강동구 천호대로 1052 엘웨딩홀,NaN,37.536831,127.128418
4,4,0.0,2,780.0,30,M,ST-1215,ST-827,2023-01-01,ST-1215,서울특별시 성북구 안암로 17,SK제삼주유소 앞,37.579449,127.024193
